In [1]:

import os
from pyspark.sql import Row, SparkSession
from typing import Dict, Any
import json
import uuid
from functools import partial
from typing import Iterable, Dict, Any
# create spark session
from pyspark.sql import Row, DataFrame
from loguru import logger
from xinghe.spark import *
from app.common.json_util import *
from llm_web_kit.extractor.extractor_chain import ExtractSimpleFactory
from xinghe.s3 import *


import uuid
import traceback
from datetime import datetime
from llm_web_kit.input.datajson import DataJson
from func_timeout import FunctionTimedOut, func_timeout

#| track_id                  | uuid                           | 全局唯一的ID                                                                                                        | 是              |
#| dataset_name              | str                            | 数据集的名字（全局唯一），这个名字是管理员输入的，然后做索引的时候带到index里来                                     | 是              |
#| data_source_category      | str                            | 这一行数据代表的是HTML，PDF，EBOOK,CC,labCC类型                                                                     | 是，此处是 HTML |
#| html                      | 字符串                         | 以UTF-8为编码的HTML文件全文                                                                                         | 是              |
#| url                       | 字符串                         | 这个文件的来源网址                                                                                                  | 是              |
#| file_bytes                | 整数                           | 文件的size, 单位是byte                                                                                              | 是              |
#| meta_info                 | 字典                           | 存放关于文件的元信息:如果能从文件里获取到作者，制作日期等信息。或者数据本身就带有一些其他的信息都放入到这个字段里。 | 是              |
#| meta_info->input_datetime | 其格式为 `yyyy-mm-dd HH:MM:SS` | 生成这个json索引文件这一条数据的时间，可以不用那么精确                                                              | 是              |


def process_platform_data(spark: SparkSession, config,platform: str, input_paths: list, version: str = "001"):
    """接收 spark 作为参数而不是持有它"""
    platform_config = platform_configs.get(platform)
    if not platform_config:
        raise ValueError(f"Unsupported platform: {platform}")

    # Driver 端操作
    input_df = read_any_path(spark, ",".join(input_paths), config)
    print(f"读取数据结束")
    # 准备 Worker 端配置
    worker_config = {
        "field_mappings": platform_config.get("field_mappings"),
        "extractor_config": platform_config.get("extractor_config")
    }
    # 分别广播不同配置
    broadcast_field_mappings = spark.sparkContext.broadcast(worker_config["field_mappings"])
    broadcast_extractor_config_path = spark.sparkContext.broadcast(worker_config["extractor_config"])  # 广播路径

    print("原始数据json结构")
    input_rdd = input_df.rdd.map(lambda x: Row(**{**json.loads(x.value), "filename": x.filename})).cache()
  #  pandas_df = input_rdd.toDF().toPandas()
  #  print(pandas_df)
    print(input_rdd.take(1)[0].asDict().keys())
    # 数据转换（仅用字段映射）
    transformed_rdd = input_rdd.map(
        lambda row: transform_row(row, broadcast_field_mappings.value)
    )
   
    print("formatter数据结束")
    # 数据抽取（仅用抽取器配置路径）
    processed_rdd = transformed_rdd.mapPartitions(
         lambda x:extract_data(
            x,broadcast_extractor_config_path = broadcast_extractor_config_path.value
        )
    )
    print("extractor数据结束")
    print("写入数据中")
   
    # 输出结果Row(value=json.dumps(x.asDict()))).toDF()
    write_any_path(processed_rdd.map(lambda x: Row(value=json.dumps(x.asDict()))).toDF(), platform_config["output_template"],config)
    print("写入数据结束")

    # 清理广播变量
    broadcast_field_mappings.unpersist()
    broadcast_extractor_config_path.unpersist()



def transform_row(row, config: dict) -> Row:
    """根据平台配置转换行数据"""
    mappings = config

    return Row(
        track_id=getattr(row, mappings["track_id"],str(uuid.uuid4())),
        url=getattr(row, mappings["url"], ''),
        html=getattr(row, mappings["html"], ''),
        page_layout_type=mappings.get("page_layout_type_map").get(
            getattr(row, mappings["layout_field"], ''),
            "article"
        ),
        domain=extract_domain_info(getattr(row, mappings["url"], ''))['domain'],
        dataset_name=mappings["dataset_name"],
        data_source_category=mappings["data_source_category"],
        meta_info={"filename":row.filename}
    )




def extract_data(partition, broadcast_extractor_config_path):
    from loguru import logger
    extractor_chain = ExtractSimpleFactory.create('/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc')
    timeout_seconds = 10
    # 为每个分区创建唯一的错误日志文件
    # partition_id = str(uuid.uuid4())
    # current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    # error_log_path = f"s3://xyz-llm-users/xyz-users/yujia/CC-MAIN-2024-33/output/v002/error_logs/{current_time}_{partition_id}.json"
    # s3_doc_writer = S3DocWriter(path=error_log_path)
    try:
        for row in partition:
            d = row.asDict()
           
            input_data = DataJson(d)
            data_e: DataJson = func_timeout(timeout_seconds, extractor_chain.extract, args=(input_data,))
            #data_e: DataJson = extractor_chain.extract(input_data)
            yield Row(**data_e.to_dict())
    except FunctionTimedOut as e1:
            d['__error'] = {
                "error_type":"TIMEOUT",
                "error_message": "extract function timeout",
                "traceback":"TIMEOUT"
            }
            yield Row(**d)
    except Exception as e:
            # 记录更详细的错误信息
            error_info = {
                "error_type": type(e).__name__,
                "error_message": str(e),
                "traceback": traceback.format_exc(),
            }
            logger.error(error_info)
            # s3_doc_writer.write(error_info)
            d['__error'] = error_info
            yield Row(**d)
           


def _safe_extract(data: Dict, extractor, timeout: int = 10) -> Dict:
  
    """直接返回字典，避免生成器"""
    try:
        timeout_seconds = 10
        input_data = DataJson(data.asDict())
        print(input_data)
        data_e: DataJson = func_timeout(timeout_seconds, extractor.extract, \
                                                args=(input_data,))

        print(Row(**data_e.to_dict()))
        return data_e.to_dict() 
    except Exception as e:
        # 记录更详细的错误信息
        error_info = {
            "error_type": type(e).__name__,
            "error_message": str(e),
            "traceback": traceback.format_exc(),
        }
        logger.error(error_info)
        # s3_doc_writer.write(error_info)
        data['__error'] = error_info
        yield Row(**d)


from urllib.parse import urlparse

def extract_domain_info(url: str) -> dict:
    """从 URL 中提取完整的域名信息"""
    parsed = urlparse(url)
    netloc = parsed.netloc
    domain_parts = netloc.split(":")
    domain = domain_parts[0]  # 去除端口号
    root_domain = ".".join(domain.split(".")[-2:]) if len(domain.split(".")) >= 2 else domain

    return {
        "full_url": url,
        "netloc": netloc,
        "domain": domain,
        "root_domain": root_domain
    }
        
def handle_error(row: Dict, error: Exception) -> Dict:
    """统一错误处理"""
    row_dict = row.asDict()
    return {
     ** row,
    "__error": {
        "type": type(error).__name__,
        "message": str(error),
        "traceback": traceback.format_exc()
    }
    }

    
def extract_platform_from_s3_path(s3_path: str) -> str:
    """
    从 S3 路径中提取平台名称（存储桶后的第一个目录）
    
    示例输入: 
    - "s3://private-cooperate-data/zh-web-baijiahao/20241218_p1/"
    输出: "zh-web-baijiahao"
    
    - "s3://private-cooperate-data/DouBan/"
    输出: "DouBan"
    """
    # 分割路径并过滤空字符串
    parts = [p for p in s3_path.split("/") if p.strip() != ""]
    
    # 验证路径格式
    if len(parts) < 3:
        raise ValueError(f"无效的 S3 路径格式: {s3_path}")
    
    # 平台名称是存储桶后的第一个目录
    return parts[2]



config = {
    "spark_conf_name": "spark_4",
    "skip_success_check": True,
    "spark.executor.memory":"8g",
    # 根据个人路径进行替换1
    "spark.executorEnv.LLM_WEB_KIT_CFG_PATH": "/share/jiangwenhao/.llm-web-kit.jsonc",
    "spark.dynamicAllocation.maxExecutors": "400"
}
paths = [
#"s3://private-cooperate-data/zh-web-baijiahao/",
#"s3://private-crawl-data/zh-web-netease/20241218_p1/",
   "s3://private-crawl-data/zh-web-tencent/20241218_p1/",
    "s3://private-crawl-data/zh-web-sohu/20241218_p1/",
 #   "s3://private-crawl-data/zh-web-sina/20241218_p1/",
   # "s3://crawl-data/blog_sina_com_cn/gz_file/1729501052/",
   # "s3://private-cooperate-data/DouBan/"
    

]


# spark = new_spark_session("llm_kit_cc", config)
# version="008"
# for path in paths:
#     platform = extract_platform_from_s3_path(path)
#     print(f"路径: {path} → 平台: {platform}")
#     platform_configs = {
#
#     "zh-web-baijiahao": {
#
#         "field_mappings": {
#             "track_id":"track_id",
#             "page_layout_type_map" :{
#                                 "":"article",
#                                 "文章":"article",
#                                 "网易":'article',
#                                 "视频":"video"
#                             },
#         "dataset_name": "baijiahao",
#         "url":"url",
#         "html":"content",
#         "layout_field": "channel",
#         "data_source_category":"JSON"
#         },
#         "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
#         "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
#
#     },
#     "zh-web-netease": {
#
#         "field_mappings": {
#              "track_id":"track_id",
#             "page_layout_type_map" :{
#                                 "":"article",
#                                 "文章":"article",
#                                 "网易":'article',
#                                 "视频":"video"
#                             },
#         "dataset_name": "net-ease",
#         "url":"url",
#         "html":"content",
#         "layout_field": "f_name",
#         "data_source_category":"HTML"
#         },
#         "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
#         "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
#     },
#     "zh-web-tencent": {
#
#         "field_mappings": {
#              "track_id":"track_id",
#             "page_layout_type_map" :{
#                                 "":"article",
#                                 "文章":"article",
#                                 "网易":'article',
#                                 "视频":"video",
#                                 "腾讯网":"article"
#                             },
#         "dataset_name": "tencent",
#         "url":"url",
#         "html":"content",
#         "layout_field": "f_name",
#         "data_source_category":"HTML"
#         },
#         "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
#         "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
#     },
#      "zh-web-sohu": {
#
#         "field_mappings": {
#              "track_id":"track_id",
#             "page_layout_type_map" :{
#                                 "":"article",
#                                 "文章":"article",
#                                 "网易":'article',
#                                 "视频":"video",
#                                 "搜狐网":"article"
#                             },
#         "dataset_name": "souhu",
#         "url":"url",
#         "html":"content",
#         "layout_field": "f_name",
#         "data_source_category":"HTML"
#         },
#         "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
#         "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
#     },
#     "zh-web-sina": {
#
#         "field_mappings": {
#              "track_id":"track_id",
#             "page_layout_type_map" :{
#                                 "":"article",
#                                 "文章":"article",
#                                 "网易":'article',
#                                 "视频":"video",
#                                 "搜狐网":"article",
#                                 "黑猫投诉":"forum"
#                             },
#         "dataset_name": "sina",
#         "url":"url",
#         "html":"content",
#         "layout_field": "f_name",
#         "data_source_category":"HTML"
#         },
#         "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
#         "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
#     },
#
#     "blog_sina_com_cn": {
#
#         "field_mappings": {
#              "track_id":"track_id",
#             "page_layout_type_map" :{
#                                 "":"article",
#                                 "文章":"article",
#                                 "网易":'article',
#                                 "视频":"video",
#                                 "搜狐网":"article",
#                                 "黑猫投诉":"forum"
#                             },
#         "dataset_name": "sina",
#         "url":"url",
#         "html":"html",
#         "layout_field": "f_name",
#         "data_source_category":"HTML"
#         },
#         "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
#         "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
#     },
#     "DouBan": {
#
#         "field_mappings": {
#              "track_id":"track_id",
#             "page_layout_type_map" :{
#                                 "":"article",
#                                 "文章":"article",
#                                 "网易":'article',
#                                 "视频":"video",
#                                 "搜狐网":"article",
#                                 "黑猫投诉":"forum",
#                                 "豆瓣网":""
#                             },
#         "dataset_name": "sina",
#         "url":"url",
#         "html":"html",
#         "layout_field": "f_name",
#         "data_source_category":"HTML"
#         },
#         "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
#         "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
#     },
#}
    # 处理正式数据
 #  process_platform_data(
 #      spark=spark,
 #      config=config,
 #      platform=platform,
 #      input_paths=[path],
 #      version=version
 # )


    # 处理其他平台数据
    # processor.process_platform_data("other_platform", [...])

In [2]:

import os
from pyspark.sql import Row, SparkSession
from typing import Dict, Any
import json
import uuid
from functools import partial
from typing import Iterable, Dict, Any
# create spark session
from pyspark.sql import Row, DataFrame
from loguru import logger
from xinghe.spark import *
from app.common.json_util import *

from xinghe.s3 import *


import uuid
import traceback
from datetime import datetime
from llm_web_kit.input.datajson import DataJson
from func_timeout import FunctionTimedOut, func_timeout

In [7]:
config = {
    "spark_conf_name": "spark_4",
    "skip_success_check": True,
    # 根据个人路径进行替换1
    "spark.executorEnv.LLM_WEB_KIT_CFG_PATH": "/share/jiangwenhao/.llm-web-kit.jsonc",
    "spark.dynamicAllocation.maxExecutors": "400",
}
spark = new_spark_session("llm_kit_cc", config)


In [14]:
input_paths =[f's3://zhuanxiang-hw60p/article/souhu/v002/part-67d16a86edd3-000008.jsonl']





# Driver 端操作
input_df = read_any_path(spark, ",".join(input_paths), config)

In [15]:
input_df.rdd.map(lambda x: Row(**json.loads(x.value))).take(1)[0].asDict()

{'track_id': '9d46fc71-f681-4e56-8a23-fa17b7e78840',
 'url': 'https://www.sohu.com/a/617284846_120664110',
 'html': '<p data-role="original-title" style="display:none">原标题：望岳谈｜浙江江苏包机出国招商抢订单，山东也没闲着</p> \n<!--                                          --> \n<p>望岳谈｜浙江江苏包机出国招商抢订单，山东也没闲着 </p> \n<p><img img_width="1280" img_height="400" data-src="4sWCR8BxipzMtuwIic3zeSI0m71znpbbVePc/zHb+MqKotkA+qg6URaOyuEKJ46568QcvkubkNnnwWAWNFf88on9Ge3CfJ1KG1a5nxFKNbM=" src="https://www.sohu.com/a/4sWCR8BxipzMtuwIic3zeSI0m71znpbbVePc/zHb+MqKotkA+qg6URaOyuEKJ46568QcvkubkNnnwWAWNFf88on9Ge3CfJ1KG1a5nxFKNbM="></p> \n<p><strong><span>□刘芝杰</span></strong></p> \n<p><span>这两天，“浙江万家企业包机出国抢订单”“苏州包机出国抢订单”等话题和视频火爆全网，网友纷纷点赞。</span></p> \n<p><span>据央视网报道，宁波市组织商务包机2批8班次，累计服务外贸人员600人次，完成订单额20亿美金。截至今年11月底，宁波市外贸企业派出850多人分别赴美国、德国和中东欧等地开展商务活动。</span></p> \n<p><img img_width="1080" img_height="782" data-src="lrZM2CeEj0WTHFVM4IzFdiI0m71znpbbVePc/zHb+Mr0lSmkCbmeC2nKaHOroMmn6tTXafC+Ea+qwy1POySbraHFNSqPdpLtWqFrYm5DdqU=" src="https:/

In [99]:
version="018"
# sub_dir = "202401"
input_paths =[f's3://private-crawl-data/zh-web-tencent/20241218_p1/2021/1733722748975.json.gz']
platform = extract_platform_from_s3_path(input_paths[0])
platform_configs = {

    "zh-web-baijiahao": {

        "field_mappings": {
            "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video"
                            },
        "dataset_name": "baijiahao",
        "url":"url",
        "html":"content",
        "layout_field": "channel",
        "data_source_category":"JSON"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/{sub_dir}/v{version}/"

    },
    "zh-web-netease": {

        "field_mappings": {
             "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video"
                            },
        "dataset_name": "net-ease",
        "url":"url",
        "html":"content",
        "layout_field": "f_name",
        "data_source_category":"HTML"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
    },
    "zh-web-tencent": {

        "field_mappings": {
             "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video",
                                "腾讯网":"article"
                            },
        "dataset_name": "tencent",
        "url":"url",
        "html":"content",
        "layout_field": "f_name",
        "data_source_category":"HTML"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
    },
     "zh-web-sohu": {

        "field_mappings": {
             "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video",
                                "搜狐网":"article"
                            },
        "dataset_name": "souhu",
        "url":"url",
        "html":"content",
        "layout_field": "f_name",
        "data_source_category":"HTML"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
    },
    "zh-web-sina": {

        "field_mappings": {
             "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video",
                                "搜狐网":"article",
                                "黑猫投诉":"forum"
                            },
        "dataset_name": "sina",
        "url":"url",
        "html":"content",
        "layout_field": "f_name",
        "data_source_category":"HTML"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
    },

    "blog_sina_com_cn": {

        "field_mappings": {
             "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video",
                                "搜狐网":"article",
                                "黑猫投诉":"forum"
                            },
        "dataset_name": "sina",
        "url":"url",
        "html":"html",
        "layout_field": "f_name",
        "data_source_category":"HTML"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
    },
    "DouBan": {
       
        "field_mappings": {
             "track_id":"track_id",
            "page_layout_type_map" :{
                                "":"article",
                                "文章":"article",
                                "网易":'article',
                                "视频":"video",
                                "搜狐网":"article",
                                "黑猫投诉":"forum",
                                "豆瓣网":""
                            },
        "dataset_name": "sina",
        "url":"url",
        "html":"html",
        "layout_field": "f_name",
        "data_source_category":"HTML"
        },
        "extractor_config": "/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc",
        "output_template": f"s3://llm-users-phdd2/jiangwenhao/article/{platform}/v{version}/"
    },
}
platform_config = platform_configs.get(platform)
platform_config


{'field_mappings': {'track_id': 'track_id',
  'page_layout_type_map': {'': 'article',
   '文章': 'article',
   '网易': 'article',
   '视频': 'video',
   '腾讯网': 'article'},
  'dataset_name': 'tencent',
  'url': 'url',
  'html': 'content',
  'layout_field': 'f_name',
  'data_source_category': 'HTML'},
 'extractor_config': '/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc',
 'output_template': 's3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/'}

In [100]:

config = {
    "spark_conf_name": "spark_4",
    "skip_success_check": True,
    # 根据个人路径进行替换1
    "spark.executorEnv.LLM_WEB_KIT_CFG_PATH": "/share/jiangwenhao/.llm-web-kit.jsonc",
    "spark.dynamicAllocation.maxExecutors": "400",
}
spark = new_spark_session("llm_kit_cc", config)


print(f"路径: {input_paths[0]} → 平台: {platform}")

platform = "zh-web-tencent"
"""接收 spark 作为参数而不是持有它"""
platform_config = platform_configs.get(platform)
if not platform_config:
    raise ValueError(f"Unsupported platform: {platform}")

# Driver 端操作
input_df = read_any_path(spark, ",".join(input_paths), config)
print(f"读取数据结束")
# 准备 Worker 端配置
worker_config = {
    "field_mappings": platform_config.get("field_mappings"),
    "extractor_config": platform_config.get("extractor_config")
}
# 分别广播不同配置
broadcast_field_mappings = spark.sparkContext.broadcast(worker_config["field_mappings"])
broadcast_extractor_config_path = spark.sparkContext.broadcast(worker_config["extractor_config"])  # 广播路径


路径: s3://private-crawl-data/zh-web-tencent/20241218_p1/2021/1733722748975.json.gz → 平台: zh-web-tencent
读取数据结束


In [101]:
get_s3_config("s3://private-crawl-data/zh-web-tencent/20241218_p1/")


{'endpoint': 'http://10.140.97.35',
 'ak': 'OA8AZVFH6110XW4A51NX',
 'sk': 'aZ5XfNpTZ8xSAa9lcs7MhYAy7wGr3WAzhzO5AfR8'}

In [102]:
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
#安全解析函数（包含异常捕获）
def safe_json_loads(s):
    try:
        json.loads(s)
        return True
    except:
        return False
# 将Python函数注册为Spark UDF
safe_json_udf = udf(safe_json_loads, BooleanType())

# 在filter中使用UDF生成的Column表达式
df_filtered = input_df.filter(safe_json_udf(col("value")))



In [103]:

input_rdd = df_filtered.rdd.map(lambda x: Row(**{**json.loads(x.value), "filename": x.filename})).cache()


In [106]:

input_rdd.filter(lambda x : x['url'] == 'https://new.qq.com/omn/20210619/20210619A06HAY00').take(1)[0]

Row(posttime='2021-06-19 16:28:28', n_content_county=[], m_identity='5135396', n_origin='', n_headimg_url='http://qqpublic.qpic.cn/qq_public/0/0-4116853528-6548C7FB88720DFC9FA3F8F9909FE09E/0?fmt=jpg&size=27&h=803&w=900&ppv=1', f_domain_sec='new.qq.com', title='21例！“疯狂”的中国疫苗在新加坡上演！自费也疯抢！1天上千人打爆咨询热线～', uuid='b99eec121b09da3606feb815b4ee4af5', content='万事通说 好消息！千呼万唤，中国科兴疫苗终于可以接种了~ 万事通即时新闻 今日疫情数据 新增21人 社区病例14人 客工宿舍起 7起境外输入病例 累计62403人 疫情爆发以来，坡岛网友们除了每天紧盯新增的确诊病例数据，就是关注着各国新冠疫苗的研发进程，翘首期盼着疫苗的早日到来，似乎这样才能吃一颗定心丸~ 终于在去年的12月21日晚七点半左右，新加坡航班运载的首批辉瑞疫苗在樟宜机场着陆了。 大家都知道，新加坡一共购买了3款疫苗，在获批使用辉瑞（Pfizer）—BioNTech和莫德纳（Moderna）两款冠病疫苗后，在今年的2月23日，中国的科兴疫苗也到货了。 网友支持中国疫苗 希望尽快获批使用 广大通心粉听到中国科兴疫苗到了，也按捺不住激动的心情！纷纷表达了对科兴疫苗的关注！ 还有网友表示，科兴疫苗已被多国批准使用，很多国家的领导人都带头接种了这款疫苗，大家应该对它有信心。 但是过了这么久，中国科兴疫苗还是没有消息。 万事通收到网友超多的询问 自新冠疫苗抵达新加坡后，询问度最高的也是中国疫苗，万事通每天都会在后台收到大量通心粉的留言，内容都是灵魂拷问：新加坡到底啥时候才能通过中国的科兴疫苗的审核？啥时候才能接种中国的科兴疫苗？ 再不通过审批 科兴疫苗就要过期了 大家都知道2021年2月23日，中国科兴疫苗就已经运抵新加坡了，但是一直到3个月后，科兴疫苗仍然未批准使用。 对于迟迟不批准科兴疫苗这事儿，新加坡卫生部表示还在评估科兴疫苗的安全和有效性，还在等待疫苗制造商呈交更多数

In [107]:

def transform_row(row, config: dict) -> Row:
    """根据平台配置转换行数据"""
    mappings = config

    return Row(
        track_id=getattr(row, mappings["track_id"],str(uuid.uuid4())),
        url=getattr(row, mappings["url"], ''),
        html=getattr(row, mappings["html"], ''),
        page_layout_type=mappings.get("page_layout_type_map").get(
            getattr(row, mappings["layout_field"], ''),
            "article"
        ),
        domain=extract_domain_info(getattr(row, mappings["url"], ''))['domain'],
        dataset_name=mappings["dataset_name"],
        data_source_category=mappings["data_source_category"],
        meta_info={"filename":row.filename}
    )
#  pandas_df = input_rdd.toDF().toPandas()
#  print(pandas_df)

# 数据转换（仅用字段映射）
transformed_rdd = input_rdd.map(
    lambda row: transform_row(row, broadcast_field_mappings.value)
).repartition(1000)


In [108]:
# transformed_rdd_1 = transformed_rdd.filter(lambda x: "<!--VIDEO_0-->"  not in x.html)
# transformed_rdd_2 = transformed_rdd_1.filter(lambda x: "<!--MUSIC_0-->" not in x.html)
# transformed_rdd_2 = transformed_rdd_2.cache()
transformed_rdd.cache()
# 18660
# 62
# 50000-(18660+62)=31278

MapPartitionsRDD[19] at coalesce at NativeMethodAccessorImpl.java:0

In [117]:


def extract_data(partition, broadcast_extractor_config_path):
   
    from loguru import logger

    extractor_chain = ExtractSimpleFactory.create('/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc')
    timeout_seconds = 10
    # 为每个分区创建唯一的错误日志文件
    # partition_id = str(uuid.uuid4())
    # current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    # error_log_path = f"s3://xyz-llm-users/xyz-users/yujia/CC-MAIN-2024-33/output/v002/error_logs/{current_time}_{partition_id}.json"
    # s3_doc_writer = S3DocWriter(path=error_log_path)
    for row in partition:
        try:

            d = row.asDict()
            input_data = DataJson(d)
            data_e: DataJson = func_timeout(timeout_seconds, extractor_chain.extract, args=(input_data,))
            #data_e: DataJson = extractor_chain.extract(input_data)
          
            yield Row(**c)
        except FunctionTimedOut as e1:
                d['__error'] = {
                    "error_type":"TIMEOUT",
                    "error_message": "extract function timeout",
                    "traceback":"TIMEOUT"
                }

                yield Row(**d)
        except Exception as e:
                # 记录更详细的错误信息
                error_info = {
                    "error_data":Row(**d),
                    "error_type": type(e).__name__,
                    "error_message": str(e),
                    "traceback": traceback.format_exc(),
                }
                logger.error(error_info)
                # s3_doc_writer.write(error_info)
                d['__error'] = error_info

                yield Row(**d)
           


# 数据抽取（仅用抽取器配置路径）
processed_rdd = transformed_rdd.mapPartitions(
     lambda x:extract_data(
        x,broadcast_extractor_config_path = broadcast_extractor_config_path.value
    )
)



In [118]:
processed_rdd.filter(lambda x : x['url'] == 'https://new.qq.com/omn/20210619/20210619A06HAY00').take(1)[0]


Row(track_id='2b3bfe18-621b-4a8c-8479-f5714bf49244', url='https://new.qq.com/omn/20210619/20210619A06HAY00', html='万事通说 好消息！千呼万唤，中国科兴疫苗终于可以接种了~ 万事通即时新闻 今日疫情数据 新增21人 社区病例14人 客工宿舍起 7起境外输入病例 累计62403人 疫情爆发以来，坡岛网友们除了每天紧盯新增的确诊病例数据，就是关注着各国新冠疫苗的研发进程，翘首期盼着疫苗的早日到来，似乎这样才能吃一颗定心丸~ 终于在去年的12月21日晚七点半左右，新加坡航班运载的首批辉瑞疫苗在樟宜机场着陆了。 大家都知道，新加坡一共购买了3款疫苗，在获批使用辉瑞（Pfizer）—BioNTech和莫德纳（Moderna）两款冠病疫苗后，在今年的2月23日，中国的科兴疫苗也到货了。 网友支持中国疫苗 希望尽快获批使用 广大通心粉听到中国科兴疫苗到了，也按捺不住激动的心情！纷纷表达了对科兴疫苗的关注！ 还有网友表示，科兴疫苗已被多国批准使用，很多国家的领导人都带头接种了这款疫苗，大家应该对它有信心。 但是过了这么久，中国科兴疫苗还是没有消息。 万事通收到网友超多的询问 自新冠疫苗抵达新加坡后，询问度最高的也是中国疫苗，万事通每天都会在后台收到大量通心粉的留言，内容都是灵魂拷问：新加坡到底啥时候才能通过中国的科兴疫苗的审核？啥时候才能接种中国的科兴疫苗？ 再不通过审批 科兴疫苗就要过期了 大家都知道2021年2月23日，中国科兴疫苗就已经运抵新加坡了，但是一直到3个月后，科兴疫苗仍然未批准使用。 对于迟迟不批准科兴疫苗这事儿，新加坡卫生部表示还在评估科兴疫苗的安全和有效性，还在等待疫苗制造商呈交更多数据，还有留意该疫苗在其他国家的使用情况。 中国科兴疫苗 被世卫批准使用 终于在今年6月1日，世界卫生组织宣布，将中国的科兴新冠疫苗列入紧急使用清单。 这就意味着，万千在新加坡网友期盼的科兴疫苗，马上就可以在新加坡接种了！ 中国科兴疫苗在新加坡的接种之路，可谓是一波三折。 全岛40多家医疗机构 争抢科兴疫苗 由于科兴疫苗缺乏完整资料，并未通过科学局的审批，因此不在国家疫苗计划里。也就是说大家想打，只能自费去私人医疗场所接种！政府宣布医疗机构可以向卫生部提出申请，如果他们获

25/03/10 23:29:38 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_19_214 !
25/03/10 23:29:38 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_19_481 !
25/03/10 23:29:38 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_19_810 !
25/03/10 23:29:38 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_19_10 !
25/03/10 23:29:38 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_19_255 !
25/03/10 23:29:38 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_19_377 !
25/03/10 23:29:38 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_19_441 !
25/03/10 23:29:38 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_19_935 !
25/03/10 23:29:38 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_19_180 !
25/03/10 23:29:38 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_19_452 !
25/03/10 23:29:38 WARN BlockManagerMasterEndpoint: 

In [69]:
result_df = processed_rdd.map(lambda x: Row(value=json_dumps(x.asDict()))).toDF()
# 输出结果Row(value=json.dumps(x.asDict()))).toDF()
#write_any_path(processed_rdd.map(lambda x: Row(value=json.dumps(x.asDict()))).toDF(), platform_config["output_template"],config)

platform_config

{'field_mappings': {'track_id': 'track_id',
  'page_layout_type_map': {'': 'article',
   '文章': 'article',
   '网易': 'article',
   '视频': 'video',
   '腾讯网': 'article'},
  'dataset_name': 'tencent',
  'url': 'url',
  'html': 'content',
  'layout_field': 'f_name',
  'data_source_category': 'HTML'},
 'extractor_config': '/share/jiangwenhao/notebooks/定向子集专项/subset-spliz.jsonc',
 'output_template': 's3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/'}

In [70]:
result_df.take(1)[0]

Row(value='{"track_id":"2341da94-5ca1-461a-83cb-82341210c02b","url":"http://auto.qq.com/a/20190125/005641.htm","html":"<p style=\\"TEXT-INDENT: 2em\\">[腾讯汽车 站]编辑从深圳市增特汽车贸易有限公司了解到，奥迪S3最高优惠4.00万元，促销时间为2019年01月26日--2019年01月26日， 欢迎有意向的朋友到店试乘试驾。</p> \\n<p><a href=\\"http://wecar.qq.com/dealer/promotion/detail/id/18500/article_id/2389346\\" target=\\"_blank\\"><img src=\\"http://p.qpic.cn/auto_comm/0/1f43339a266a3b3d581c6b6eb08ba0caca3d8b0e/0\\" /></a></p> \\n<p><a href=\\"http://data.auto.qq.com/car_public/1/disp_pic_nl.shtml#sid=1329&amp;tid=38&amp;pid=2989640\\" target=\\"_blank\\"><img src=\\"http://img1.gtimg.com/datalib_img//18-12-19/7/144267ff996e2dd22ef40ee0576fb2f9.jpg\\" /></a></p> \\n<p align=\\"center\\" style=\\"font-family: 宋体; font-size: 10pt;\\">奥迪S3外观</p> \\n<p><a href=\\"http://data.auto.qq.com/car_public/1/disp_pic_nl.shtml#sid=1329&amp;tid=39&amp;pid=2989642\\" target=\\"_blank\\"><img src=\\"http://img1.gtimg.com/datalib_img//18-12-19/2/b78d7a66c5046e95158a3f48a8138a3b.j

In [ ]:
write_any_path(result_df, platform_config["output_template"])



[Stage 3:====================>                               (400 + 600) / 1000]

In [41]:
x = "s3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/"
s3 = get_s3_client(x)
_ = list(list_s3_objects(client=s3, path = x))
_

['s3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/_SUMMARY_rows_190324275_bytes_1997223294260_files_1000',
 's3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/part-67c97ee0b90d-000000.jsonl',
 's3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/part-67c97ee0b90d-000001.jsonl',
 's3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/part-67c97ee0b90d-000002.jsonl',
 's3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/part-67c97ee0b90d-000003.jsonl',
 's3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/part-67c97ee0b90d-000004.jsonl',
 's3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/part-67c97ee0b90d-000005.jsonl',
 's3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/part-67c97ee0b90d-000006.jsonl',
 's3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/part-67c97ee0b90d-000007.jsonl',
 's3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v018/part-67c97ee0b90d-000008.jsonl',
 's3://llm-users

In [3]:
get_s3_config("s3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/2019/v017/")

{'endpoint': 'http://10.135.0.241',
 'ak': 'L5F6OE3EQEK00V4MV0E4',
 'sk': 'oAwAZurrQps6VbOXKHpG9XjRtmjAG7ROOjfzquwC'}

In [ ]:
# 清理广播变量
broadcast_field_mappings.unpersist()
broadcast_extractor_config_path.unpersist()

In [23]:
from xinghe.spark import *
from app.common.json_util import *

from xinghe.s3 import *

delete_path = "s3://llm-users-phdd2/jiangwenhao/"
client = get_s3_client(delete_path)
delete_s3_object(delete_path +'article/zh-web-baijiahao/v004', client = client, dry_run = False)

In [24]:
list(list_s3_objects('s3://llm-users-phdd2/jiangwenhao/article/zh-web-baijiahao/', client = client, recursive=False))

['s3://llm-users-phdd2/jiangwenhao/article/zh-web-baijiahao/v002/',
 's3://llm-users-phdd2/jiangwenhao/article/zh-web-baijiahao/v003/',
 's3://llm-users-phdd2/jiangwenhao/article/zh-web-baijiahao/v004/',
 's3://llm-users-phdd2/jiangwenhao/article/zh-web-baijiahao/v005/']

In [ ]:
   def process_platform_data(self, platform: str, input_paths: list, version: str = "001"):
        """处理指定平台数据的主流程"""
        # 获取平台配置
        config = self.platform_configs.get(platform)
        if not config:
            raise ValueError(f"Unsupported platform: {platform}")

        # 读取原始数据
        input_df = read_any_path(self.spark, ",".join(input_paths), config)

        # 数据转换
        transformed_rdd = input_df.rdd.map(
            partial(self._transform_row, platform=platform)
        ).repartition(6000)

        # 数据抽取
        processed_rdd = transformed_rdd.mapPartitions(
            partial(self._extract_data, platform=platform)
        )

        # 写入输出
        output_path = config["output_template"].format(
            platform=platform,
            version=version.zfill(3)
        )
        write_any_path(
            processed_rdd.map(lambda x: Row(value=json.dumps(x))).toDF(),
            output_path,
            {"skip_output_check": True}
        )

In [4]:
from pyspark.sql import Row, SparkSession
from typing import Dict, Any
import json
import uuid
from functools import partial
from typing import Iterable, Dict, Any
# create spark session
from pyspark.sql import Row, DataFrame

from xinghe.spark import *
from app.common.json_util import *

from xinghe.s3 import *


config = {
    "spark_conf_name": "spark_4",
    "skip_success_check": True,
    # 根据个人路径进行替换1
    "spark.executorEnv.LLM_WEB_KIT_CFG_PATH": "/share/jiangwenhao/.llm-web-kit.jsonc",
    "spark.dynamicAllocation.maxExecutors": "400",
}

spark = new_spark_session("llm_kit_cc", config)

processor = WebDataProcessor(spark)
platform="netease"
input_paths=["s3://private-crawl-data/zh-web-netease/20241218_p1/"]
version="002"


In [7]:
# 获取平台配置
self =processor
config = processor.platform_configs.get(platform)
if not config:
    raise ValueError(f"Unsupported platform: {platform}")

# 读取原始数据
input_df = read_any_path(self.spark, ",".join(input_paths), config)


In [21]:
input_df.count()

25/02/27 15:05:42 WARN TaskSetManager: Lost task 120.0 in stage 13.0 (TID 694) (host-10-140-92-29 executor 1): java.io.EOFException: Unexpected end of input stream
	at org.apache.hadoop.io.compress.DecompressorStream.decompress(DecompressorStream.java:165)
	at org.apache.hadoop.io.compress.DecompressorStream.read(DecompressorStream.java:105)
	at java.io.InputStream.read(InputStream.java:101)
	at org.apache.hadoop.util.LineReader.fillBuffer(LineReader.java:191)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:227)
	at org.apache.hadoop.util.LineReader.readLine(LineReader.java:185)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.nextKeyValue(LineRecordReader.java:200)
	at org.apache.spark.sql.execution.datasources.RecordReaderIterator.hasNext(RecordReaderIterator.scala:39)
	at org.apache.spark.sql.execution.datasources.HadoopFileLinesReader.hasNext(HadoopFileLinesReader.scala:67)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.

Py4JJavaError: An error occurred while calling o1376.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 120 in stage 13.0 failed 4 times, most recent failure: Lost task 120.3 in stage 13.0 (TID 697) (host-10-140-92-130 executor 2): java.io.EOFException: Unexpected end of input stream
	at org.apache.hadoop.io.compress.DecompressorStream.decompress(DecompressorStream.java:165)
	at org.apache.hadoop.io.compress.DecompressorStream.read(DecompressorStream.java:105)
	at java.io.InputStream.read(InputStream.java:101)
	at org.apache.hadoop.util.LineReader.fillBuffer(LineReader.java:191)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:227)
	at org.apache.hadoop.util.LineReader.readLine(LineReader.java:185)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.nextKeyValue(LineRecordReader.java:200)
	at org.apache.spark.sql.execution.datasources.RecordReaderIterator.hasNext(RecordReaderIterator.scala:39)
	at org.apache.spark.sql.execution.

In [24]:
sample_fraction = 0.1
if sample_fraction==1.0:
    df_sample = input_df
else:
    df_sample = input_df.sample(fraction=sample_fraction)

In [32]:
df_sample = df_sample.limit(1)
type(df_sample)

pyspark.sql.dataframe.DataFrame

In [35]:
transformed_rdd = df_sample.rdd.map(
        partial(self._transform_row, platform=platform)
    )
transformed_rdd.count()

25/02/27 15:09:16 WARN TaskSetManager: Lost task 0.0 in stage 33.0 (TID 1197) (host-10-140-92-130 executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/data/nm-local-dir/usercache/jiangwenhao/appcache/application_1738517174793_0504/container_e02_1738517174793_0504_01_000003/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/data/nm-local-dir/usercache/jiangwenhao/appcache/application_1738517174793_0504/container_e02_1738517174793_0504_01_000003/pyspark.zip/pyspark/worker.py", line 820, in process
    out_iter = func(split_index, iterator)
  File "/share/jiangwenhao/envs/code_clean_venv/lib/python3.10/site-packages/pyspark/rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/share/jiangwenhao/envs/code_clean_venv/lib/python3.10/site-packages/pyspark/rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/share/jiangwenhao/envs/code_cl

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 4 times, most recent failure: Lost task 0.3 in stage 33.0 (TID 1200) (host-10-140-92-130 executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/data/nm-local-dir/usercache/jiangwenhao/appcache/application_1738517174793_0504/container_e02_1738517174793_0504_01_000003/pyspark.zip/pyspark/worker.py", line 830, in main
    process()
  File "/data/nm-local-dir/usercache/jiangwenhao/appcache/application_1738517174793_0504/container_e02_1738517174793_0504_01_000003/pyspark.zip/pyspark/worker.py", line 820, in process
    out_iter = func(split_index, iterator)
  File "/share/jiangwenhao/envs/code_clean_venv/lib/python3.10/site-packages/pyspark/rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/share/jiang

In [ ]:
processor.platform_configs.get(platform)